In [1]:
!pip install pylatex python-dotenv PyGithub

In [2]:
import os
import uuid
import shutil
import pylatex
import pandas as pd
import lightkurve as lk 
import matplotlib.pyplot as plt

from datetime import datetime
from tqdm.notebook import tqdm
from multiprocessing import cpu_count, Pool
from pylatex.section import Chapter, Subsection
from pylatex import Document, Section, Figure, NoEscape, Command


import os
from dotenv import load_dotenv

load_dotenv()

plt.ioff()
%run pipeline.ipynb

OUTPUT_PATH = "/home/jovyan/SRMP-JMAG/output"

In [3]:
def get_sectors(tic):
    """Returns a list of available sectors given an TIC."""
    sectors = []
    for result in lk.search_tesscut("TIC " + str(tic)):
        sectors.append(int(result.mission[0].split(" ")[2]))
        
    return {"tic": tic, "sectors": sectors}

def get_targets(tics):
    """Constructs a dictionary of TICs and available sectors from a list of TICs"""
    targets = []

    with tqdm(total=len(tics)) as pbar:
        for tic in tics:
            pbar.update()
            targets.append(get_sectors(tic))
    return targets
                
targets = get_targets(pd.read_csv("./stars.csv")["TIC"][1:10])


  0%|          | 0/9 [00:00<?, ?it/s]

In [4]:
complex = {}
not_complex = {}

# This first loop will populate the list of complex and non-complex stars
for target in targets:
    for sector in target["sectors"]:
        if sector <= 5: # FFIs are only high enough res after sector 5.
            continue
        lc = load_lc(f"TIC {target['tic']}", sector)
            
        # Determine complexity
        if complexitycheck(lc):
            if target["tic"] not in complex.keys():
                complex[target["tic"]] = []
            complex[target["tic"]].append(sector)
        else:
            if target["tic"] not in not_complex.keys():
                not_complex[target["tic"]] = []
            not_complex[target["tic"]].append(sector)
        if not os.path.isdir(f"{OUTPUT_PATH}/{target['tic']}"):
            os.mkdir(f"{OUTPUT_PATH}/{target['tic']}")

        # # Plot river and save
        # plt.figure(river_plot(lc).number) # this is necessary to make the output current and save the correct figure.
        # plt.savefig(f"{OUTPUT_PATH}/{target['tic']}/{sector}_river.png")
        # plt.close()

        # # Plot lc and save
        # plt.figure(graph_lc(lc).number) 
        # plt.savefig(f"{OUTPUT_PATH}/{target['tic']}/{sector}_plot.png")
        # plt.close()

        # TODO: Periodogram when fixed.

In [5]:
def make_dataframes(l):
    
    df = pd.DataFrame()
    
    tics = []
    sectors = []
    for tic in list(l.keys()):
        for sector in l[tic]:
            tics.append(tic)
            sectors.append(sector)
    
    df["TIC"] = tics
    df["Sectors"] = sectors

    return df

make_dataframes(complex).to_csv(f"{OUTPUT_PATH}/complex.csv")
make_dataframes(not_complex).to_csv(f"{OUTPUT_PATH}/not_complex.csv")

In [6]:
from github import Github, InputGitAuthor

def get_pdf_link(token, repo_name, file_path, commit_message, file_name):
    """
    Add a file to a GitHub repository.

    Args:
    token (str): GitHub personal access token.
    repo_name (str): Repository name in the format 'username/repo'.
    file_path (str): Path to the local file to upload.
    commit_message (str): Commit message for the upload.

    Returns:
    str: URL of the committed file.
    """
    # Initialize GitHub object with token
    g = Github(token)
    repo = g.get_repo(repo_name)

    # Read file content to upload
    with open(file_path, 'rb') as file:
        content = file.read()

    # Retrieve the file path within the repository
    git_file_path = file_path.split('/')[-1]  # Assuming the file is in the root directory of the repo

    # Get commit author details (optional)
    author = InputGitAuthor(
        'Your Name', 
        'your_email@example.com'
    )

    # Upload file to repository
    git_file = repo.create_file(git_file_path, commit_message, content, author=author)
    tex_link = f"https://cdn.jsdelivr.net/gh/{repo_name}/{file_name}.zip"
    return f"https://www.overleaf.com/docs?snip_uri={tex_link}"


def make_chapter(title, data, doc):
    with doc.create(Chapter(title)):
        for tic in data.keys():
            with doc.create(Section(f"TIC {tic}")):
                for sector in data[tic]:
                    with doc.create(Subsection(f"Sector {sector}", label=f"{tic}_{sector}")):
                        lc = load_lc(f"TIC {tic}", sector)
                        
                        plt.figure(river_plot(lc).number)
                        with doc.create(Figure(position='H')) as plot:
                            doc.append(NoEscape(r'\begin{center}'))
                            plot.add_plot(width=NoEscape(r'0.75\textwidth'))
                            doc.append(NoEscape(r'\end{center}'))
                        plt.close()

                        plt.figure(graph_lc(lc).number)
                        with doc.create(Figure(position='H')) as plot:
                            doc.append(NoEscape(r'\begin{center}'))
                            plot.add_plot(width=NoEscape(r'0.75\textwidth'))
                            doc.append(NoEscape(r'\end{center}'))
                        plt.close()


def make_pdf_report():
    temp = pylatex.utils.make_temp_dir()
    try:
        shutil.rmtree("./build")
    except:
        print("Didn't delete build dir")
    os.mkdir("./build")

    doc = Document(documentclass="report", geometry_options={"margin": "0.5in"})

    doc.preamble.append(Command("title", "JMAG Complex Rotator Pipeline Results"))
    doc.preamble.append(Command("date", f"Generated on {datetime.now().strftime('%x %X')}"))
    doc.preamble.append(NoEscape(r'\makeatletter\expandafter\def\expandafter\@afterheading\expandafter{\@afterheading\mbox{}}\makeatother'))
    doc.preamble.append(NoEscape(r'\AddToHook{cmd/section/before}{\clearpage}'))
    doc.preamble.append(NoEscape(r'\AddToHook{cmd/subsection/before}{\clearpage}'))
    doc.preamble.append(NoEscape(r'\usepackage{float}'))
    doc.preamble.append(NoEscape(r'\usepackage{hyperref}'))
    doc.preamble.append(NoEscape(r'\hypersetup{colorlinks=true}'))


    doc.append(NoEscape(r'\maketitle'))
    doc.append(NoEscape(r'\tableofcontents'))

    make_chapter("Complex Rotators", complex, doc)
    make_chapter("Non-Complex Rotators", not_complex, doc)
    print(temp)
    doc.generate_tex("report")

    f = open('./report.tex', 'r')
    tex = f.read().replace(str(temp), 'figures')
    f.close()
    

    with open("./build/main.tex", "w+") as f:
        f.write(tex)

    file = uuid.uuid4()
    shutil.move(temp, "./build/")
    os.rename(f"./build/{temp.split('/')[2]}", "./build/figures")
    shutil.make_archive(f"./{file}", "zip", f"./build")
    
    print(get_pdf_link(os.environ.get("TOKEN"), 'gideongrinberg/jmag-reports', f'./{file}.zip', 'Upload build', file))
make_pdf_report()

/tmp/pylatex-tmp.vcp_q202
https://www.overleaf.com/docs?snip_uri=https://cdn.jsdelivr.net/gh/gideongrinberg/jmag-reports/0f0833aa-52a7-432c-b377-929350641000.zip
